In [145]:
import pandas as pd
import numpy as np

In [146]:
titanic = pd.read_csv('train.csv')
t = titanic.set_index(['PassengerId'])
pd.set_option('display.max_rows', None)

In [156]:
result = pd.DataFrame(index=t.columns, columns=t.columns)
for outer_column in t.columns:
    sum = 0
    if outer_column in ['Name', 'Survived']:
        continue
    ff = t[t[outer_column].duplicated(keep=False)][outer_column].value_counts()
    for i in ff.items():
        percent = round(t[t[outer_column] == i[0]].fillna(-1)['Survived'].value_counts(normalize=True)*100, 3).max()
        column_value = t[t[outer_column] == i[0]].fillna(-1)['Survived'].value_counts().sort_values(ascending=False).head(1).index[0]
        sum += i[1] * percent
    result['Survived'][outer_column] = round(sum / ff.sum(), 2)

t_result = result.dropna(axis='columns', how='all').dropna(axis='index', how='all').sort_values(by='Survived', ascending=False)
t_result

,Survived
Ticket,84.88
Cabin,82.52
Sex,78.68
Fare,77.96
Pclass,67.9
Age,65.19
Embarked,63.78
SibSp,63.3
Parch,63.03


In [163]:
def search(entry):
    for column in t_result.index:
        for value in t[column]:
            if value == entry[column]:
                percent = round(t[t[column] == value].fillna(-1)['Survived'].value_counts(normalize=True)*100, 2).max()
                column_value = t[t[column] == value].fillna(-1)['Survived'].value_counts().sort_values(ascending=False).head(1).index[0]
                return column, percent, column_value

def first_model(x):
    entries = []
    for entry in x:
        entries.append({
            'Pclass': entry[1],
            'Name': entry[2],
            'Sex': entry[3],
            'Age': entry[4],
            'SibSp': entry[5],
            'Parch': entry[6],
            'Ticket': entry[7],
            'Fare': entry[8],
            'Cabin': entry[9],
            'Embarked': entry[10]
        })

    response = []
    result = []
    for entry in entries:
        response.append(search(entry))

    for i in response:
        result.append(i[2])

    return result

In [149]:
dataset = pd.read_csv('test.csv')
dataset.head()

dataset2 = pd.read_csv('gender_submission.csv')
dataset2.head(), dataset.head()

dataset_final = dataset.merge(dataset2, on='PassengerId')

x = dataset_final.iloc[:, :-1].values
y = dataset_final.iloc[:, 11].values


In [210]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)
y_pred = first_model(x_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)

print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[75  7]
 [ 8 36]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.91      0.91        82
           1       0.84      0.82      0.83        44

    accuracy                           0.88       126
   macro avg       0.87      0.87      0.87       126
weighted avg       0.88      0.88      0.88       126

Accuracy: 0.8809523809523809
